# Atomic Agents + Opik – Context-Provider Demo



This tutorial demonstrates how to use `OpikContextProvider` to inject current trace

information into agent prompts for better observability and debugging.



**Key Concepts:**

* Context providers inject trace metadata into system prompts

* This allows agents to reference their execution context

* Useful for multi-agent workflows and debugging


**Simple Workflow:**

1. `MainAgent` – processes user requests and may call a specialist

2. `SpecialistAgent` – handles specific domain questions

3. Both agents can access the current trace ID through their context provider

## Installation

```bash

pip install "opik[atomic_agents]" atomic-agents instructor

```

In [66]:
from typing import Optional

from pydantic import Field
from instructor.client import Instructor

from atomic_agents.agents.base_agent import (
    BaseAgent as BaseChatAgent,
    BaseAgentConfig as BaseChatAgentConfig,
)
from atomic_agents.lib.base.base_io_schema import BaseIOSchema

from opik.integrations.atomic_agents import track_atomic_agents, OpikContextProvider
from opik import context_storage



And now, to activate tracing with an one-liner, we execute

In [67]:
track_atomic_agents(project_name="context-provider-demo")

## Defining an Atomic Agent

We first start by defining the input and output schemas of an agent, as illustrated in:

In [68]:
class UserQuery(BaseIOSchema):
    """User question schema"""
    message: str = Field(..., description="User question or request")


class AgentResponse(BaseIOSchema):
    """Agent response schema"""
    message: str = Field(..., description="Agent response")
    trace_context: Optional[str] = Field(None, description="Current trace context for debugging")



and then we can create a minimal agent simply via:

In [69]:
from atomic_agents.agents.base_agent import BaseAgentConfig


class MainAgent(BaseChatAgent):
    """Main agent"""
    
    input_schema = UserQuery
    output_schema = AgentResponse

dummy_instructor_client = Instructor(
    client=None,
    create=lambda **kwargs: UserQuery(chat_message="Echo: Hello Atomic Agents!"),
)

agent = MainAgent(
    config=BaseChatAgentConfig(model="gpt-3.5-turbo", client=dummy_instructor_client),
)

We then switch our attention to demonstrating what is the information that the context provider injects to the system prompt.

In [ ]:
def demonstrate_context_provider():
    """Demonstrate context provider functionality with mock trace data"""
    
    print("=== OpikContextProvider Demonstration ===\n")
    
    print("1. Basic Context Provider:")
    basic_provider = OpikContextProvider(project_name="demo-project")
    print(f"   Without trace_id: '{basic_provider.get_info()}'\n")
    
    basic_provider.set_trace_id("trace-12345")
    print(f"   With trace_id: \n'{basic_provider.get_info()}'")
    
    print("\n2. Context Provider with Metadata:")
    advanced_provider = OpikContextProvider(
        project_name="demo-project",
        trace_id="6vsfr1sf1r490",
        include_metadata=True,
        metadata={
            "agent_role": "specialist",
            "workflow_type": "delegation",
            "user_id": "user-123"
        }
    )
    print(f"   Full context:\n{advanced_provider.get_info()}")
    
    print("\n3. System Prompt Injection Example:")
    print("   When an agent uses this context provider, this information")
    print("   gets automatically injected into the system prompt:")
    print("   ```")
    print(f"   {advanced_provider.get_info()}")
    print("   ```")

    return basic_provider, advanced_provider

_, advanced_provider = demonstrate_context_provider()

=== OpikContextProvider Demonstration ===

1. Basic Context Provider:
   Without trace_id: ''

   With trace_id: 
'[Opik Trace Info]
trace_id: trace-12345
project:  demo-project'

2. Context Provider with Metadata:
   Full context:
[Opik Trace Info]
trace_id: 6vsfr1sf1r490
project:  demo-project
agent_role: specialist
workflow_type: delegation
user_id: user-123

3. System Prompt Injection Example:
   When an agent uses this context provider, this information
   gets automatically injected into the system prompt:
   ```
   [Opik Trace Info]
trace_id: 6vsfr1sf1r490
project:  demo-project
agent_role: specialist
workflow_type: delegation
user_id: user-123
   ```


Finally, as per the Atomic Agents API, we may enable a context provider in the following way:

In [ ]:

agent.register_context_provider("opik_context", advanced_provider)

NameError: name 'context_provider' is not defined